In [3]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
import os
import string
import numpy as np
import pandas as pd

from collections import Counter
from keras.layers import Embedding
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding



from os import listdir
from collections import Counter

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


ModuleNotFoundError: No module named 'tensorflow'

# Przygotowanie plików

In [ ]:
df = pd.read_excel('Reviews.xlsx')
df

,user_review,user_suggestion
0,I'm scared and hearing creepy voices. So I'll...,1
1,"Best game, more better than Sam Pepper's YouTu...",1
2,"A littly iffy on the controls, but once you kn...",1
3,"Great game, fun and colorful and all that.A si...",1
4,Not many games have the cute tag right next to...,1
...,...,...
17489,Arguably the single greatest mmorp that exists...,1
17490,"An older game, to be sure, but has its own cha...",1
17491,When I frist started playing Everquest 2 it wa...,1
17492,cool game. THe only thing that REALLY PISSES M...,1


In [ ]:
df = pd.read_excel('Reviews.xlsx')

# tworzenie folderów na recenzje
if not os.path.exists('positive'):
    os.makedirs('positive')
if not os.path.exists('negative'):
    os.makedirs('negative')

pos_file_num = 1
neg_file_num = 1

# pętla po rzędach
for index, row in df.iterrows():
    review = row['user_review']
    sentiment = row['user_suggestion']
    
    # zapis jednej recenzji do osobnego pliku .txt i wrzucenie jej do odpowiedniego folderu
    if sentiment == 1:
        filename = f"positive/positive-{pos_file_num}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(review)
        pos_file_num += 1
    else:
        filename = f"negative/negative-{neg_file_num}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(review)
        neg_file_num += 1

## Pliki podzielone na test i trening ręcznie = test_positive ~1090 plików, test_negative ~750

# Przetwarzanie tekstu

In [ ]:
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8')
    text = file.read()
    file.close()
    return text

#czyszczenie dokumentu
def clean_doc(doc):
 #podziel przez spacje
    tokens = doc.split()
 #usuń interpunkcję
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
 #usuń niealfabetyczne tokeny
    tokens = [word for word in tokens if word.isalpha()]
 #usuń stop wordy
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
 #usuń tokeny mniejsze niż 2 znaki
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

def add_doc_to_vocab(filename, vocab):
    doc = load_doc(filename)
    tokens = clean_doc(doc)
    vocab.update(tokens)

#przejście przez foldery
def process_docs(directory, vocab):
    for filename in listdir(directory):
        path = directory + '/' + filename
        add_doc_to_vocab(path, vocab)

vocab = Counter()
#ścieżki folderów
process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\positive', vocab)
process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\negative', vocab)
# wielkość korpusu
print(len(vocab))
# najczęstrze słowa w korpusie
print(vocab.most_common(50))

82896
[('game', 44091), ('like', 10393), ('get', 10175), ('play', 9928), ('The', 8236), ('time', 6402), ('good', 6153), ('fun', 5894), ('one', 5744), ('dont', 5736), ('Access', 5314), ('even', 5233), ('really', 5211), ('Early', 5153), ('games', 4825), ('would', 4328), ('playing', 4324), ('new', 4198), ('much', 4121), ('people', 4045), ('free', 4017), ('This', 4003), ('money', 3881), ('players', 3814), ('want', 3671), ('make', 3534), ('still', 3497), ('played', 3299), ('You', 3239), ('great', 3232), ('It', 3201), ('hours', 3163), ('better', 2969), ('pay', 2947), ('Its', 2904), ('also', 2840), ('lot', 2838), ('way', 2723), ('If', 2695), ('many', 2679), ('cant', 2564), ('buy', 2526), ('need', 2514), ('every', 2510), ('first', 2491), ('well', 2484), ('see', 2404), ('go', 2389), ('say', 2364), ('system', 2325)]


In [ ]:
#usń z korpusu tokeny które występują mniej niż 3 razy
min_occurane = 3
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

19131


In [ ]:
vocab = set(tokens)
vocab

{'level',
 'Feats',
 'bes',
 'Decent',
 'enjoyement',
 'hattori',
 'stale',
 'armorweapons',
 'early',
 'Having',
 'separates',
 'mainline',
 'communications',
 'glance',
 'psychology',
 'Fortnight',
 'Fires',
 'chilling',
 'subscribed',
 'gaem',
 'gimics',
 'rewardsHonor',
 'Demons',
 'examples',
 'ginormous',
 'AGAIN',
 'OYUN',
 'magic',
 'manually',
 'Indar',
 'Theyve',
 'macro',
 'Modes',
 'unbalanced',
 'moreIf',
 'hive',
 'experimented',
 'flaming',
 'stat',
 'sequences',
 'Yellow',
 'ESL',
 'Slot',
 'boy',
 'picks',
 'stars',
 'providing',
 'committed',
 'dumpster',
 'largely',
 'Download',
 'maintained',
 'presumably',
 'Forgive',
 'Cryptics',
 'CONNECTION',
 'stil',
 'Aviation',
 'pad',
 'defensive',
 'fun',
 'hey',
 'Microtransactions',
 'gravity',
 'infused',
 'blizzard',
 'wil',
 'HoTS',
 'Stream',
 'vendor',
 'editing',
 'Admittedly',
 'heard',
 'Unfortunately',
 'printed',
 'misunderstanding',
 'pins',
 'Exile',
 'judgement',
 'dash',
 'transfering',
 'cautious',
 'improp

# Trenowanie modelu

In [ ]:
def train_clean_doc(doc, vocab):
 # split into tokens by white space
    tokens = doc.split()
 # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
 # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

# load all docs in a directory
def process_docs(directory, vocab, isTrain):
   documents = list()
 # walk through all files in the folder
   for filename in listdir(directory):
 # create the full path of the file to open
      path = directory + '/' + filename
 # load the doc
      doc = load_doc(path)
 # clean doc
      tokens = train_clean_doc(doc, vocab)
 # add to list
      documents.append(tokens)
   return documents
 
# load all training reviews
positive_docs = process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\positive', vocab, True)
negative_docs = process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\negative', vocab, True)
train_docs = negative_docs + positive_docs


In [ ]:
# stwórz tokenizację
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_docs)

# encodig 'dokumentów' do modelu sekwencyjnego
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [ ]:
# padding pustymi _tokenami_
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

max_length

1600

In [ ]:
#przypisywanie snetymentu do określonych wierszy (train_docs = neg + pos)
ytrain = np.array([0 for _ in range(6775)] + [1 for _ in range(8872)])

# Uwaga!!!!

## parametry range() w ytest zalezą od tego ile plików wydzielono ręcznie z oryginalnie przetworzonych plików __positive__ i __negative__ do plików __"test_negative" i __"test_positive"__ w tym przypadku ręcznie wydzielono 752 negatywne i 1095 pozytyznych recenzji do z danych _treningowych_

In [ ]:
positive_docs = process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\test_positive', vocab, False)
negative_docs = process_docs(r'C:\Users\sjkli\Desktop\Rafinacja\test_negative', vocab, False)
test_docs = negative_docs + positive_docs
# text na liczby
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# wyrównywanie do największej długości jednej recenzji
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# etykiety częsci testowej 750 negatywnych i 1095 pozytywnych
ytest = np.array([0 for _ in range(752)] + [1 for _ in range(1095)])

In [ ]:
# wielkość (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

14907

# Sam model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1600, 100)         1490700   
                                                                 
 conv1d (Conv1D)             (None, 1593, 32)          25632     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 796, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 25472)             0         
                                                                 
 dense (Dense)               (None, 10)                254730    
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                        

## Trenowanie (parametry typu epoki optimizer wybrane są na podstawie podręcznika)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(Xtrain, ytrain, epochs=10, verbose=2)

Epoch 1/10
489/489 - 32s - loss: 0.5321 - accuracy: 0.7552 - 32s/epoch - 66ms/step
Epoch 2/10
489/489 - 32s - loss: 0.3488 - accuracy: 0.8905 - 32s/epoch - 66ms/step
Epoch 3/10
489/489 - 32s - loss: 0.2434 - accuracy: 0.9335 - 32s/epoch - 65ms/step
Epoch 4/10
489/489 - 31s - loss: 0.1775 - accuracy: 0.9570 - 31s/epoch - 64ms/step
Epoch 5/10
489/489 - 31s - loss: 0.1431 - accuracy: 0.9672 - 31s/epoch - 63ms/step
Epoch 6/10
489/489 - 31s - loss: 0.1196 - accuracy: 0.9728 - 31s/epoch - 63ms/step
Epoch 7/10
489/489 - 31s - loss: 0.1088 - accuracy: 0.9744 - 31s/epoch - 63ms/step
Epoch 8/10
489/489 - 31s - loss: 0.1012 - accuracy: 0.9757 - 31s/epoch - 64ms/step
Epoch 9/10
489/489 - 31s - loss: 0.0981 - accuracy: 0.9755 - 31s/epoch - 63ms/step
Epoch 10/10
489/489 - 30s - loss: 0.1073 - accuracy: 0.9709 - 30s/epoch - 61ms/step


# Ewaluacja modelu

In [ ]:
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 73.957771
